# Entrenamiento de un Clasificador Naive Bayes

En esta hoja de trabajo veremos cómo entrenar un clasificador Naive Bayes y evaluar su rendimiento. Empezaremos importando las librerías y funciones que utilizaremos más adelante.


In [1304]:
import pandas as pd
import csv
import numpy as np
np.random.seed(357823)
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

# Lectura de datos y creación de los atributos y el objetivo

Vamos a empezar por importar el dataset con todos los atributos que calculamos para la realización de este trabajo. Luego, separamos el dataset en atributos por los cuales Naive Bayes se va a entrenar. Finalmente, elegiremos el objetivo, que en este caso es un objetivo categórico binario.

In [1305]:
# Lectura del csv para la tabla con todas las métricas
nodes = pd.read_csv("../tablas/tableWithAllAtributes.csv")

# Lectura del csv para la tabla con todos los atributos sin los de agrupamiento
nodesWithoutClustering = pd.read_csv("../tablas/tableWithoutClustering.csv")

# Lectura del csv para la tabla con todos los atributos sin los de comunidades
nodesWithoutCommunity = pd.read_csv("../tablas/tableWithoutCommunity.csv")

#Tablas con todos los atributos sin los de nucleos
nodesWithoutKernel = pd.read_csv("../tablas/tableWithoutKernel.csv")


# 1. Tabla con todos los atributos

In [1306]:
nodes.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


## 1.1. Tratamiento de los datos

Es necesario codificar cualquier columna categórica que el modelo Naive Bayes no pueda manejar directamente. Utilizaremos la codificación adecuada para convertir las categorías en números, discretizaremos algunos valores para trabajar mejor y normalizaremos estos números si es necesario.

In [1307]:
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos = nodes.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [1308]:
# Objetivo
objetivo = nodes['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

In [1309]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)


# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)


# Aplicamos la normalización solo a la columna 'name'
atributos['name'] = normalizador.fit_transform(atributos[['name']])

print()
print("///Tabla con la columna name codificada y normalizada///")
atributos.head()


///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']

///Tabla con la columna name codificada y normalizada///


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,0.929866,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,0.106687,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,0.191517,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,0.969442,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [1310]:
# Discretizamos:
# Discretizamos usando uniform ya que si usamos la estrategia de quantile se eliminan datos por ser los intervalos demasiado pequeños
discretizador = KBinsDiscretizer(
    n_bins=700,  # Hemos 700 intervalos ya que es como mejor rendimiento saca para los valores que hemos probado
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='uniform'  # Intervalos de igual tamaño
    )

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_discretizados = atributos.copy()

atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
atributos_discretizados.head()

# El id_node no lo discretizamos, al igual que el name


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.929866,0.0,281.0,0.0,124.0,70.0,0.0,1.0,106.0,0.0
2,2,0.106687,0.0,220.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.191517,0.0,251.0,0.0,0.0,18.0,0.0,3.0,63.0,0.0
4,4,0.969442,0.0,186.0,0.0,0.0,0.0,0.0,7.0,21.0,0.0


## 1.2. Elección de hiperparámetros

Vamos a elegir los mejores hiperparámetros para entrenar nuestro clasificador Naive Bayes. Para ello, vamos a utilizar la búsqueda en rejilla (Grid Search), donde vamos a combinar diferentes valores para el parámetro de suavizado (alpha). Probaremos con valores de alpha como 1, 2, 3, 4, 5,..., 10.

Como verás más adelante, el mejor rendimiento se obtiene con un valor de alpha de 1.

In [1311]:
tubería_NB = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB())])
rejilla_de_hiperparámetros = {
    # Suavizado del estimador llamado naive_Bayes
    'naive_Bayes__alpha': range(1, 11)
}

In [1312]:
búsqueda_en_rejilla = GridSearchCV(tubería_NB,
                                   rejilla_de_hiperparámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_discretizados,objetivo)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesador',
                                        KBinsDiscretizer(encode='ordinal',
                                                         n_bins=700,
                                                         strategy='uniform')),
                                       ('naive_Bayes', CategoricalNB())]),
             param_grid={'naive_Bayes__alpha': range(1, 11)}, scoring='recall')

In [1313]:
búsqueda_en_rejilla.best_params_ # Observamos que el valor 1 para el Suavizado de Laplace es la mejor opción

{'naive_Bayes__alpha': 1}

In [1314]:
búsqueda_en_rejilla.best_score_

0.7260724362721616

## 1.3. Validación por retención

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados. Dividiremos el dataset en datos de entrenamiento y datos de prueba. En este caso, usaremos un 80% de los datos para entrenar y un 20% para probar. Entrenaremos el clasificador Naive Bayes con diferentes configuraciones de hiperparámetros y evaluaremos su rendimiento.


In [1315]:
# Dividimos los conjuntos de prueba y entrenamiento
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_discretizados, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

In [1316]:
clasificador_NB1 = CategoricalNB(alpha=1)  # alpha es el parámetro de suavizado
clasificador_NB1.fit(atributos_entrenamiento, objetivo_entrenamiento)

CategoricalNB(alpha=1)

In [1317]:
predicciones = clasificador_NB1.predict(atributos_prueba)
predicciones

array([0., 0., 0., ..., 0., 1., 0.])

In [1318]:
cm = confusion_matrix(objetivo_prueba, predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Positivo", "Verdadero Negativo"], columns=["Predicho Positivo", "Predicho Negativo"])
print(tabla_cm)

                    Predicho Positivo  Predicho Negativo
Verdadero Positivo               4749                843
Verdadero Negativo                850               1098


In [1319]:
recall_score(objetivo_prueba, predicciones)

0.5636550308008214

## 1.4. Validación cruzada

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados mediante validación cruzada. Dividiremos el dataset en varios subconjuntos y realizaremos múltiples iteraciones de entrenamiento y prueba, asegurándonos de que cada subconjunto se use una vez como conjunto de prueba y las restantes como conjunto de entrenamiento. Utilizaremos validación cruzada con `k` pliegues, donde `k` es el número de subconjuntos en los que dividiremos nuestros datos. Esto nos permitirá evaluar el rendimiento del clasificador Naive Bayes de manera más robusta y fiable.

Calcularemos la validación cruizada para un alpha óptimo (alpha = 1) y para un alpha no óptimo y comprobaremos que el rendimiento es superior con el alpha óptimo.

In [1320]:
# Con alpha = 1 que es el mejor valor
tubería_NB1 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=1))])

In [1321]:
# Con alpha != 1 
tubería_NB2 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=2))])

In [1322]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo = cross_validate(tubería_NB1,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo

{'fit_time': array([0.03942633, 0.03800583, 0.03116512, 0.03211904, 0.03161049,
        0.03056097, 0.03146195, 0.03209281, 0.03135109, 0.03084588]),
 'score_time': array([0.00697803, 0.00561881, 0.0060606 , 0.00608587, 0.00598788,
        0.00714278, 0.00519633, 0.00598383, 0.00498557, 0.00601435]),
 'test_score': array([0.94552929, 0.67659138, 0.68172485, 0.65195072, 0.68069815,
        0.66016427, 0.70020534, 0.66427105, 0.65297741, 0.94661191])}

In [1323]:
# Validación cruzada para Alpha NO óptimo (Alpha != 1)
resultados_validación_cruzadaConAlphaNoOptimo = cross_validate(tubería_NB2,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaNoOptimo

{'fit_time': array([0.03332734, 0.02970719, 0.03512192, 0.03469777, 0.03269434,
        0.03189135, 0.0306356 , 0.03137112, 0.02984524, 0.03148389]),
 'score_time': array([0.00595593, 0.00601244, 0.00598407, 0.00698137, 0.00498533,
        0.00598383, 0.00522637, 0.00520062, 0.00598049, 0.00621986]),
 'test_score': array([0.90030832, 0.62422998, 0.61601643, 0.5862423 , 0.62320329,
        0.61396304, 0.63244353, 0.60985626, 0.59753593, 0.89322382])}

In [1324]:
resultados_validación_cruzadaConAlphaOptimo['test_score'].mean()

0.7260724362721616

In [1325]:
resultados_validación_cruzadaConAlphaNoOptimo['test_score'].mean()

0.6697022903824198

## 1.5. Análisis del resultado

Al analizar nuestros resultados, observamos que el mejor puntaje se logra al establecer un valor de alpha igual a 1. Esto nos proporciona una confianza del 72.61%. Dado que en nuestros datos el 75% corresponden a desarrolladores web y el 25% a desarrolladores de IA, al examinar la matriz de confusión, concluimos que el árbol de decisión está mejor entrenado que un clasificador aleatorio. Aunque hay una cantidad significativa de falsos positivos, esto podría ser atribuible a la escasez de ejemplos de desarrolladores de IA en la partición de los datos, lo que sugiere que el modelo tiene un rendimiento aceptable dadas las condiciones de partición.

# 2. Rendimiento del modelo según las métricas añadidas
Ahora vamos a observar qué métricas son más importantes, en cuanto a rendimiento se refiere, para el entrenamiento del modelo de Naive Bayes.

## 2.1. Tabla con todos los atributos menos los de agrupamiento

In [1326]:
nodesWithoutClustering.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.002227,0.151515,0.0
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.004454,0.090909,0.0
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.011136,0.030303,0.0


### 2.1.1. Tratamiento de los datos

Es necesario codificar cualquier columna categórica que el modelo Naive Bayes no pueda manejar directamente. Utilizaremos la codificación adecuada para convertir las categorías en números, discretizaremos algunos valores para trabajar mejor y normalizaremos estos números si es necesario.

In [1327]:
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos = nodesWithoutClustering.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.0
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.002227,0.151515,0.0
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.0
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.004454,0.090909,0.0
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.011136,0.030303,0.0


In [1328]:
# Objetivo
objetivo = nodesWithoutClustering['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

In [1329]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)


# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)


# Aplicamos la normalización solo a la columna 'name'
atributos['name'] = normalizador.fit_transform(atributos[['name']])

print()
print("///Tabla con la columna name codificada y normalizada///")
atributos.head()

///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']

///Tabla con la columna name codificada y normalizada///


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.0
1,1,0.929866,0.000212,0.294956,1.149733e-06,0.002227,0.151515,0.0
2,2,0.106687,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.0
3,3,0.191517,0.000133,0.278718,5.316292e-05,0.004454,0.090909,0.0
4,4,0.969442,0.000053,0.243084,6.134318e-09,0.011136,0.030303,0.0


In [1330]:
# Discretizamos:
# Discretizamos usando uniform ya que si usamos la estrategia de quantile se eliminan datos por ser los intervalos demasiado pequeños
discretizador = KBinsDiscretizer(
    n_bins=700,  # Hemos 700 intervalos ya que es como mejor rendimiento saca para los valores que hemos probado
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='uniform'  # Intervalos de igual tamaño
    )

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_discretizados = atributos.copy()

atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
atributos_discretizados.head()

# El id_node no lo discretizamos, al igual que el name


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.0,245.0,0.0,0.0,0.0,0.0
1,1,0.929866,0.0,281.0,0.0,1.0,106.0,0.0
2,2,0.106687,0.0,220.0,0.0,0.0,0.0,0.0
3,3,0.191517,0.0,251.0,0.0,3.0,63.0,0.0
4,4,0.969442,0.0,186.0,0.0,7.0,21.0,0.0


### 2.1.2. Elección de hiperparámetros

Vamos a elegir los mejores hiperparámetros para entrenar nuestro clasificador Naive Bayes. Para ello, vamos a utilizar la búsqueda en rejilla (Grid Search), donde vamos a combinar diferentes valores para el parámetro de suavizado (alpha). Probaremos con valores de alpha como 1, 2, 3, 4, 5,..., 10.

Como verás más adelante, el mejor rendimiento se obtiene con un valor de alpha de 1.

In [1331]:
tubería_NB = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB())])
rejilla_de_hiperparámetros = {
    # Suavizado del estimador llamado naive_Bayes
    'naive_Bayes__alpha': range(1, 11)
}

In [1332]:
búsqueda_en_rejilla = GridSearchCV(tubería_NB,
                                   rejilla_de_hiperparámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_discretizados,objetivo)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesador',
                                        KBinsDiscretizer(encode='ordinal',
                                                         n_bins=700,
                                                         strategy='uniform')),
                                       ('naive_Bayes', CategoricalNB())]),
             param_grid={'naive_Bayes__alpha': range(1, 11)}, scoring='recall')

In [1333]:
búsqueda_en_rejilla.best_params_ # Observamos que el valor 1 para el Suavizado de Laplace es la mejor opción

{'naive_Bayes__alpha': 1}

In [1334]:
búsqueda_en_rejilla.best_score_

0.7241237224359557

### 2.1.3. Validación por retención

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados. Dividiremos el dataset en datos de entrenamiento y datos de prueba. En este caso, usaremos un 80% de los datos para entrenar y un 20% para probar. Entrenaremos el clasificador Naive Bayes con diferentes configuraciones de hiperparámetros y evaluaremos su rendimiento.

In [1335]:
# Dividimos los conjuntos de prueba y entrenamiento
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_discretizados, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

In [1336]:
clasificador_NB1 = CategoricalNB(alpha=1)  # alpha es el parámetro de suavizado
clasificador_NB1.fit(atributos_entrenamiento, objetivo_entrenamiento)

CategoricalNB(alpha=1)

In [1337]:
predicciones = clasificador_NB1.predict(atributos_prueba)
predicciones

array([0., 0., 0., ..., 0., 0., 1.])

In [1338]:
cm = confusion_matrix(objetivo_prueba, predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Positivo", "Verdadero Negativo"], columns=["Predicho Positivo", "Predicho Negativo"])
print(tabla_cm)

                    Predicho Positivo  Predicho Negativo
Verdadero Positivo               5035                557
Verdadero Negativo                896               1052


In [1339]:
recall_score(objetivo_prueba, predicciones)

0.5400410677618069

### 2.1.4. Validación cruzada

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados mediante validación cruzada. Dividiremos el dataset en varios subconjuntos y realizaremos múltiples iteraciones de entrenamiento y prueba, asegurándonos de que cada subconjunto se use una vez como conjunto de prueba y las restantes como conjunto de entrenamiento. Utilizaremos validación cruzada con `k` pliegues, donde `k` es el número de subconjuntos en los que dividiremos nuestros datos. Esto nos permitirá evaluar el rendimiento del clasificador Naive Bayes de manera más robusta y fiable.

Calcularemos la validación cruizada para un alpha óptimo (alpha = 1) y para un alpha no óptimo y comprobaremos que el rendimiento es superior con el alpha óptimo.

In [1340]:
# Con alpha = 1 que es el mejor valor
tubería_NB1 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=1))])

In [1341]:
# Con alpha != 1 
tubería_NB2 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=2))])

In [1342]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo = cross_validate(tubería_NB1,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo

{'fit_time': array([0.02750826, 0.02447462, 0.03206658, 0.0240829 , 0.02401876,
        0.02430701, 0.02428269, 0.02346635, 0.02330971, 0.02340102]),
 'score_time': array([0.00480628, 0.00554967, 0.00622153, 0.00511265, 0.00505781,
        0.00502324, 0.00523448, 0.00517559, 0.00522685, 0.00495744]),
 'test_score': array([0.96505653, 0.66940452, 0.66632444, 0.63552361, 0.6724846 ,
        0.65400411, 0.68788501, 0.66427105, 0.65400411, 0.97227926])}

In [1343]:
# Validación cruzada para Alpha NO óptimo (Alpha != 1)
resultados_validación_cruzadaConAlphaNoOptimo = cross_validate(tubería_NB2,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaNoOptimo

{'fit_time': array([0.02517796, 0.02504086, 0.02434063, 0.02456379, 0.024261  ,
        0.02529621, 0.0243082 , 0.02417278, 0.02449608, 0.02469754]),
 'score_time': array([0.00522614, 0.00421834, 0.00616908, 0.0050149 , 0.00498581,
        0.00476027, 0.00501513, 0.00501561, 0.00545168, 0.00500774]),
 'test_score': array([0.93216855, 0.62422998, 0.61806982, 0.59342916, 0.63141684,
        0.62320329, 0.63244353, 0.6211499 , 0.59856263, 0.94147844])}

In [1344]:
resultados_validación_cruzadaConAlphaOptimo['test_score'].mean()

0.7241237224359557

In [1345]:
resultados_validación_cruzadaConAlphaNoOptimo['test_score'].mean()

0.6816152123768864

### 2.1.5. Análisis del resultado
Observamos un rendimiento del 72,41%, por tanto, este ha bajado un poco respecto la tabla con todos los atributos pero no ha bajado en exceso, por lo que consideramos que estas métricas no son muy influyentes respecto al rendimiento del modelo.

## 2.2. Tabla con todos los atributos menos los de comunidad

In [1346]:
nodesWithoutCommunity.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,Core number
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.151515
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.090909
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.030303


### 2.2.1. Tratamiento de los datos

Es necesario codificar cualquier columna categórica que el modelo Naive Bayes no pueda manejar directamente. Utilizaremos la codificación adecuada para convertir las categorías en números, discretizaremos algunos valores para trabajar mejor y normalizaremos estos números si es necesario.

In [1347]:
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','Core number']
atributos = nodesWithoutCommunity.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,Core number
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.151515
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.090909
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.030303


In [1348]:
# Objetivo
objetivo = nodesWithoutCommunity['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

In [1349]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)


# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)


# Aplicamos la normalización solo a la columna 'name'
atributos['name'] = normalizador.fit_transform(atributos[['name']])

print()
print("///Tabla con la columna name codificada y normalizada///")
atributos.head()


///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']

///Tabla con la columna name codificada y normalizada///


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,Core number
0,0,0.061673,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1,1,0.929866,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.151515
2,2,0.106687,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000
3,3,0.191517,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.090909
4,4,0.969442,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.030303


In [1350]:
# Discretizamos:
# Discretizamos usando uniform ya que si usamos la estrategia de quantile se eliminan datos por ser los intervalos demasiado pequeños
discretizador = KBinsDiscretizer(
    n_bins=700,  # Hemos 700 intervalos ya que es como mejor rendimiento saca para los valores que hemos probado
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='uniform'  # Intervalos de igual tamaño
    )

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_discretizados = atributos.copy()

atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
atributos_discretizados.head()

# El id_node no lo discretizamos, al igual que el name


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,Core number
0,0,0.061673,0.0,245.0,0.0,0.0,0.0,0.0,0.0
1,1,0.929866,0.0,281.0,0.0,124.0,70.0,0.0,106.0
2,2,0.106687,0.0,220.0,0.0,0.0,0.0,0.0,0.0
3,3,0.191517,0.0,251.0,0.0,0.0,18.0,0.0,63.0
4,4,0.969442,0.0,186.0,0.0,0.0,0.0,0.0,21.0


### 2.2.2. Elección de hiperparámetros

Vamos a elegir los mejores hiperparámetros para entrenar nuestro clasificador Naive Bayes. Para ello, vamos a utilizar la búsqueda en rejilla (Grid Search), donde vamos a combinar diferentes valores para el parámetro de suavizado (alpha). Probaremos con valores de alpha como 1, 2, 3, 4, 5,..., 10.

Como verás más adelante, el mejor rendimiento se obtiene con un valor de alpha de 1.

In [1351]:
tubería_NB = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB())])
rejilla_de_hiperparámetros = {
    # Suavizado del estimador llamado naive_Bayes
    'naive_Bayes__alpha': range(1, 11)
}

In [1352]:
búsqueda_en_rejilla = GridSearchCV(tubería_NB,
                                   rejilla_de_hiperparámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_discretizados,objetivo)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesador',
                                        KBinsDiscretizer(encode='ordinal',
                                                         n_bins=700,
                                                         strategy='uniform')),
                                       ('naive_Bayes', CategoricalNB())]),
             param_grid={'naive_Bayes__alpha': range(1, 11)}, scoring='recall')

In [1353]:
búsqueda_en_rejilla.best_params_ # Observamos que el valor 1 para el Suavizado de Laplace es la mejor opción

{'naive_Bayes__alpha': 1}

In [1354]:
búsqueda_en_rejilla.best_score_

0.6614932753122817

### 2.2.3. Validación por retención

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados. Dividiremos el dataset en datos de entrenamiento y datos de prueba. En este caso, usaremos un 80% de los datos para entrenar y un 20% para probar. Entrenaremos el clasificador Naive Bayes con diferentes configuraciones de hiperparámetros y evaluaremos su rendimiento.

In [1358]:
# Dividimos los conjuntos de prueba y entrenamiento
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_discretizados, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

In [1359]:
clasificador_NB1 = CategoricalNB(alpha=1)  # alpha es el parámetro de suavizado
clasificador_NB1.fit(atributos_entrenamiento, objetivo_entrenamiento)

CategoricalNB(alpha=1)

In [1360]:
predicciones = clasificador_NB1.predict(atributos_prueba)
predicciones

array([1., 0., 0., ..., 0., 1., 0.])

In [1361]:
cm = confusion_matrix(objetivo_prueba, predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Positivo", "Verdadero Negativo"], columns=["Predicho Positivo", "Predicho Negativo"])
print(tabla_cm)

                    Predicho Positivo  Predicho Negativo
Verdadero Positivo               4198               1394
Verdadero Negativo               1068                880


In [1362]:
recall_score(objetivo_prueba, predicciones)

0.4517453798767967

### 2.2.4. Validación cruzada

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados mediante validación cruzada. Dividiremos el dataset en varios subconjuntos y realizaremos múltiples iteraciones de entrenamiento y prueba, asegurándonos de que cada subconjunto se use una vez como conjunto de prueba y las restantes como conjunto de entrenamiento. Utilizaremos validación cruzada con `k` pliegues, donde `k` es el número de subconjuntos en los que dividiremos nuestros datos. Esto nos permitirá evaluar el rendimiento del clasificador Naive Bayes de manera más robusta y fiable.

Calcularemos la validación cruizada para un alpha óptimo (alpha = 1) y para un alpha no óptimo y comprobaremos que el rendimiento es superior con el alpha óptimo.

In [1363]:
# Con alpha = 1 que es el mejor valor
tubería_NB1 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=1))])

In [1364]:
# Con alpha != 1 
tubería_NB2 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=2))])

In [1365]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo = cross_validate(tubería_NB1,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo

{'fit_time': array([0.03276491, 0.02812052, 0.02802801, 0.0292325 , 0.02763438,
        0.02700019, 0.02919292, 0.0271163 , 0.02610636, 0.02663493]),
 'score_time': array([0.00522876, 0.00498652, 0.00505877, 0.00598407, 0.00615215,
        0.00611281, 0.00506139, 0.0050745 , 0.00498676, 0.00498676]),
 'test_score': array([0.94450154, 0.59240246, 0.58213552, 0.58829569, 0.5862423 ,
        0.57494867, 0.63655031, 0.58829569, 0.57802875, 0.94353183])}

In [1366]:
# Validación cruzada para Alpha NO óptimo (Alpha != 1)
resultados_validación_cruzadaConAlphaNoOptimo = cross_validate(tubería_NB2,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaNoOptimo

{'fit_time': array([0.03291249, 0.02792382, 0.03103995, 0.03390932, 0.05984068,
        0.06834984, 0.02692842, 0.02507377, 0.02712512, 0.02756763]),
 'score_time': array([0.0069809 , 0.0050633 , 0.00715542, 0.00598431, 0.00897527,
        0.02587247, 0.00498605, 0.00604224, 0.00417709, 0.00495648]),
 'test_score': array([0.89516958, 0.52772074, 0.5349076 , 0.52464066, 0.51950719,
        0.50616016, 0.56673511, 0.52874743, 0.50616016, 0.88398357])}

In [1367]:
resultados_validación_cruzadaConAlphaOptimo['test_score'].mean()

0.6614932753122817

In [1368]:
resultados_validación_cruzadaConAlphaNoOptimo['test_score'].mean()

0.5993732206959571

### 2.2.5. Análisis de resultado

Observamos un rendimiento del 66,15%, por tanto, aquí si ha bajado considerablemente el rendimiento del modelo respecto a la tabla con todos los atributos, por tanto, concluímos que las métricas de comunidad son muy influyentes respecto al rendimiento del modelo.

## 2.3. Tabla con todos los atributos menos los de núcleo

In [1369]:
nodesWithoutKernel.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.0
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.0
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.0


### 2.3.1. Tratamiento de los datos

Es necesario codificar cualquier columna categórica que el modelo Naive Bayes no pueda manejar directamente. Utilizaremos la codificación adecuada para convertir las categorías en números, discretizaremos algunos valores para trabajar mejor y normalizaremos estos números si es necesario.

In [1370]:
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','greedy_modularity_communities','asyn_lpa_communities']
atributos = nodesWithoutKernel.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,asyn_lpa_communities
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.0
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.0
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.0


In [1371]:
# Objetivo
objetivo = nodesWithoutKernel['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

In [1372]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)


# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)


# Aplicamos la normalización solo a la columna 'name'
atributos['name'] = normalizador.fit_transform(atributos[['name']])

print()
print("///Tabla con la columna name codificada y normalizada///")
atributos.head()

///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']

///Tabla con la columna name codificada y normalizada///


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,asyn_lpa_communities
0,0,0.061673,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
1,1,0.929866,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.0
2,2,0.106687,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
3,3,0.191517,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.0
4,4,0.969442,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.0


In [1373]:
# Discretizamos:
# Discretizamos usando uniform ya que si usamos la estrategia de quantile se eliminan datos por ser los intervalos demasiado pequeños
discretizador = KBinsDiscretizer(
    n_bins=700,  # Hemos 700 intervalos ya que es como mejor rendimiento saca para los valores que hemos probado
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='uniform'  # Intervalos de igual tamaño
    )

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_discretizados = atributos.copy()

atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
atributos_discretizados.head()

# El id_node no lo discretizamos, al igual que el name


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,asyn_lpa_communities
0,0,0.061673,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.929866,0.0,281.0,0.0,124.0,70.0,0.0,1.0,0.0
2,2,0.106687,0.0,220.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.191517,0.0,251.0,0.0,0.0,18.0,0.0,3.0,0.0
4,4,0.969442,0.0,186.0,0.0,0.0,0.0,0.0,7.0,0.0


### 2.3.2. Elección de hiperparámetros

Vamos a elegir los mejores hiperparámetros para entrenar nuestro clasificador Naive Bayes. Para ello, vamos a utilizar la búsqueda en rejilla (Grid Search), donde vamos a combinar diferentes valores para el parámetro de suavizado (alpha). Probaremos con valores de alpha como 1, 2, 3, 4, 5,..., 10.

Como verás más adelante, el mejor rendimiento se obtiene con un valor de alpha de 1.

In [1374]:
tubería_NB = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB())])
rejilla_de_hiperparámetros = {
    # Suavizado del estimador llamado naive_Bayes
    'naive_Bayes__alpha': range(1, 11)
}

In [1375]:
búsqueda_en_rejilla = GridSearchCV(tubería_NB,
                                   rejilla_de_hiperparámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_discretizados,objetivo)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesador',
                                        KBinsDiscretizer(encode='ordinal',
                                                         n_bins=700,
                                                         strategy='uniform')),
                                       ('naive_Bayes', CategoricalNB())]),
             param_grid={'naive_Bayes__alpha': range(1, 11)}, scoring='recall')

In [1376]:
búsqueda_en_rejilla.best_params_ # Observamos que el valor 1 para el Suavizado de Laplace es la mejor opción

{'naive_Bayes__alpha': 1}

In [1377]:
búsqueda_en_rejilla.best_score_

0.7246363308297334

### 2.3.3. Validación por retención

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados. Dividiremos el dataset en datos de entrenamiento y datos de prueba. En este caso, usaremos un 80% de los datos para entrenar y un 20% para probar. Entrenaremos el clasificador Naive Bayes con diferentes configuraciones de hiperparámetros y evaluaremos su rendimiento.

In [1378]:
# Dividimos los conjuntos de prueba y entrenamiento
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_discretizados, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

In [1379]:
clasificador_NB1 = CategoricalNB(alpha=1)  # alpha es el parámetro de suavizado
clasificador_NB1.fit(atributos_entrenamiento, objetivo_entrenamiento)

CategoricalNB(alpha=1)

In [1380]:
predicciones = clasificador_NB1.predict(atributos_prueba)
predicciones

array([1., 0., 1., ..., 0., 0., 0.])

In [1381]:
cm = confusion_matrix(objetivo_prueba, predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Positivo", "Verdadero Negativo"], columns=["Predicho Positivo", "Predicho Negativo"])
print(tabla_cm)

                    Predicho Positivo  Predicho Negativo
Verdadero Positivo               4886                706
Verdadero Negativo                888               1060


In [1382]:
recall_score(objetivo_prueba, predicciones)

0.5441478439425051

### 2.3.4. Validación cruzada

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados mediante validación cruzada. Dividiremos el dataset en varios subconjuntos y realizaremos múltiples iteraciones de entrenamiento y prueba, asegurándonos de que cada subconjunto se use una vez como conjunto de prueba y las restantes como conjunto de entrenamiento. Utilizaremos validación cruzada con `k` pliegues, donde `k` es el número de subconjuntos en los que dividiremos nuestros datos. Esto nos permitirá evaluar el rendimiento del clasificador Naive Bayes de manera más robusta y fiable.

Calcularemos la validación cruizada para un alpha óptimo (alpha = 1) y para un alpha no óptimo y comprobaremos que el rendimiento es superior con el alpha óptimo.

In [1383]:
# Con alpha = 1 que es el mejor valor
tubería_NB1 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=1))])

In [1384]:
# Con alpha != 1 
tubería_NB2 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=2))])

In [1385]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo = cross_validate(tubería_NB1,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo

{'fit_time': array([0.03511357, 0.03318477, 0.03007793, 0.03103209, 0.0291357 ,
        0.02892256, 0.02803469, 0.02988935, 0.02731681, 0.02836609]),
 'score_time': array([0.00698686, 0.00498629, 0.00598383, 0.00611401, 0.00598359,
        0.00517917, 0.00601411, 0.00501466, 0.00502038, 0.00598502]),
 'test_score': array([0.95786228, 0.67043121, 0.66529774, 0.64373717, 0.67864476,
        0.66427105, 0.69712526, 0.6550308 , 0.65195072, 0.96201232])}

In [1386]:
# Validación cruzada para Alpha NO óptimo (Alpha != 1)
resultados_validación_cruzadaConAlphaNoOptimo = cross_validate(tubería_NB2,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaNoOptimo

{'fit_time': array([0.03191495, 0.03250742, 0.03030777, 0.02902293, 0.02892208,
        0.02808523, 0.02692771, 0.02609158, 0.02716851, 0.02617526]),
 'score_time': array([0.00797844, 0.00605559, 0.00598431, 0.00606132, 0.00498676,
        0.00498676, 0.00700927, 0.00601339, 0.00520158, 0.00516081]),
 'test_score': array([0.91675231, 0.60882957, 0.60780287, 0.57700205, 0.62525667,
        0.60882957, 0.63655031, 0.59958932, 0.58829569, 0.9137577 ])}

In [1387]:
resultados_validación_cruzadaConAlphaOptimo['test_score'].mean()

0.7246363308297334

In [1388]:
resultados_validación_cruzadaConAlphaNoOptimo['test_score'].mean()

0.668266607013597

### 2.3.5. Análisis del resultado
Observamos un rendimiento del 72,46%, por tanto, este ha bajado un poco respecto la tabla con todos los atributos pero no ha bajado en exceso, por lo que consideramos que la métrica de núcleo no es muy influyente respecto al rendimiento del modelo.